<a href="https://colab.research.google.com/github/AchrafAsh/gnn-linear-receptive-fields/blob/main/bottlenecks_of_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys
import os.path as osp
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [3]:
import networkx as nx
import torch
import torch_geometric as tg
from torch_geometric.datasets import Planetoid, TUDataset

In [48]:
G_karate = nx.karate_club_graph()

In [4]:
path = osp.join(os.getcwd(), 'data', 'Cora')
dataset = Planetoid(path, 'Cora')
# path = osp.join(os.getcwd(), 'data', 'ENZYMES')
# dataset = TUDataset(path, name='ENZYMES')

Processing...
Done!


In [5]:
G = dataset[0]
print(G.x)
print(G.edge_index)
print(G.y)
print(G['train_mask'])
print(G)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
tensor([3, 4, 4,  ..., 3, 3, 3])
tensor([ True,  True,  True,  ..., False, False, False])
Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])


In [53]:
# MAD - Mean Average Distance (global)
def mean_average_distance(x, mask=None):
    '''
    Computes cosine similarity between each pair of node representations
    Args:
        x (torch.tensor): node features
        mask (torch.tensor): mask matrix composed of 0 and 1 only if the node pair (i,j) is the target one.
    '''

    MAD_num = 0
    MAD_den = 0

    for i in range(x.size(0)):
        D_avg_num = 0
        D_avg_den = 0
        for j in range(i):
            cos_sim = 0
            if mask == None or (mask != None and  mask[i, j]):
                cos_sim = 1 - torch.dot(x[i,:], x[j,:]) / (torch.norm(x[i, :]) * torch.norm(x[j, :]) + 1e-16)
            D_avg_num += cos_sim
            D_avg_den += 1 if cos_sim > 0 else 0
        
        if D_avg_den:
            D_avg = D_avg_num / D_avg_den
            MAD_num += D_avg_num
            MAD_den += 1 if D_avg > 0 else 0

    MAD = MAD_num / MAD_den
    return MAD

In [54]:
mean_average_distance(x=G.x, mask=tg.utils.to_dense_adj(G.edge_index)[0])

tensor(2.1652)

In [55]:
mean_average_distance(x=torch.tensor(nx.attr_matrix(G_karate)[0]),
                      mask=nx.adjacency_matrix(G_karate))

tensor(2.3117, dtype=torch.float64)

In [56]:
1 - tg.utils.to_dense_adj(G.edge_index)[0]

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 0., 1.]])

In [57]:
tg.utils.to_dense_adj(G.edge_index)[0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.]])

In [59]:
def mean_average_distance_gap(x, adj_matrix):
    '''
    Computes MADGap: MAD_remote - MAD_neighbors
    Args:
        x (torch.tensor): input feature matrix
        adj_matrix (torch.tensor): adjacency matrix
    '''

    # TODO: understand what it means to apply a mask in mean_average_distance
    MAD_rem = mean_average_distance(x, mask=1-adj_matrix)
    MAD_neb = mean_average_distance(x, mask=adj_matrix)

    return MAD_rem - MAD_neb

In [67]:
mean_average_distance_gap(x=torch.tensor(nx.attr_matrix(G_karate)[0]),
                          adj_matrix=torch.tensor(nx.adjacency_matrix(G_karate).todense()))

tensor(9.9893, dtype=torch.float64)

In [ ]:
def influence_distribution(x, h):
    '''
    Computes influence distribution
    ...math: I_x(y) = e^T \left[ \frac{ \partial h_x^{(k)} }{ \partial h_y^{(0)} } \right] e / \left( \sum_{z \in V} e^T \left[ \frac{ \partial h_x^{(k)} }{ \partial h_z^{(0)} } \right] e \right)
    
    Args:
        h (torch.tensor): hidden feature matrix
        x (torch.tensor): input feature matrix
    '''
    e = torch.ones(x.size(1), 1)

    jacob_xy = 
    I[x,y] = torch.dot( torch.dot(torch.transpose(e, 0, 1), jacob_xy), e )


In [77]:
e = torch.ones((1, 5))

e, torch.transpose(e, 0, 1)

(tensor([[1., 1., 1., 1., 1.]]), tensor([[1.],
         [1.],
         [1.],
         [1.],
         [1.]]))

In [ ]:
from torch_geometric.nn.models import JumpingKnowledge

# Bottleneck of GNNs

- [ ] 